## Combining unity VR data with imaging data
Temporal alignment using recrded microscope frame trigger signal in unity log

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from unityvr.preproc import logproc
from unityvr.viz import viz

#### Load preprocessed VR and imaging data

In [2]:
preprocDir = '../../../../data/preproc/EPG/SS96_x_7f/f01'
rawTiff = 'SS96-x-7f_f01_3-5d_EB_2LM_00002.tif'
rawLog = 'Log_2021-02-03_18-25-38.json'


In [ ]:
uvrTest_loaded = logproc.loadUVRData(preprocDir, savename)
uvrTest_loaded.printMet1aadata(1a)